<b>Name: Arpit Aggarwal</b> <br>
<b>UID: 116747189</b>

# 1. Packages

In [314]:
# header files
import numpy as np
import torch
import h5py
from matplotlib import pyplot as plt
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Cat vs Non-Cat Image Classification

# 2. Loading Dataset

Using hw2.ipynb load_data() function. The load_data() function loads data from the training and testing files. Next step, is to flatten the image so that they can be fed as an input to the neural network. Lastly, the training and testing data is normalized between 0 and 1 which will be used for the neural network.

In [297]:
def load_data(train_file, test_file):
    # Load the training data
    train_dataset = h5py.File(train_file, 'r')
    
    # Separate features(x) and labels(y) for training set
    train_set_x_orig = np.array(train_dataset['train_set_x'])
    train_set_y_orig = np.array(train_dataset['train_set_y'])

    # Load the test data
    test_dataset = h5py.File(test_file, 'r')
    
    # Separate features(x) and labels(y) for training set
    test_set_x_orig = np.array(test_dataset['test_set_x'])
    test_set_y_orig = np.array(test_dataset['test_set_y'])
    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((test_set_y_orig.shape[0]))
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

# training and testing files
train_file = "data/train_catvnoncat.h5"
test_file = "data/test_catvnoncat.h5"
train_x_orig, train_output, test_x_orig, test_output, classes = load_data(train_file, test_file) 
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1)
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1)

# Standardize data to have feature values between 0 and 1.
train_input = train_x_flatten / 255.
test_input = test_x_flatten / 255.

# print data length
print ("train_input's shape: " + str(train_input.shape))
print ("test_input's shape: " + str(test_input.shape))

train_input's shape: (209, 12288)
test_input's shape: (50, 12288)


# 3. Convert dataset to Tensor form

Convert the dataset to Tensor form so that it can be fed into the PyTorch neural network.

In [298]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# use torch.from_numpy() to get the tensor form of the numpy array
train_input = torch.from_numpy(train_input).float().to(device)
train_output = torch.from_numpy(train_output).float().to(device)
test_input = torch.from_numpy(test_input).float().to(device)
test_output = torch.from_numpy(test_output).float().to(device)

# 4. Hyper-parameters

Set the hyper-parameters of the two-layer neural net.

In [305]:
learning_rate = 0.01
num_hidden_neurons = 40
num_epochs = 3000

# 5. Model-Architecture

The model-architecture is defined using pytorch Net class. The __init__ function is where we define the architecture of the neural network, i.e in this it is two layers. The forward function is where the forward pass step of the neural network takes place.

In [306]:
# neural network class
class Net(torch.nn.Module):
    # init function
    def __init__(self, num_input_neurons, num_hidden_neurons, num_output_neurons):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(num_input_neurons, num_hidden_neurons)
        self.fc2 = torch.nn.Linear(num_hidden_neurons, num_output_neurons)
        
    # forward pass step of the neural network
    def forward(self, input):
        output = torch.nn.functional.sigmoid(self.fc2(torch.nn.functional.relu(self.fc1(input))))
        return output
    
# get the neural net object
net = Net(int(train_input.shape[1]), int(num_hidden_neurons), 1).to(device)
print(net)

Net(
  (fc1): Linear(in_features=12288, out_features=40, bias=True)
  (fc2): Linear(in_features=40, out_features=1, bias=True)
)


# 7. Loss function

We will use Binary Cross-entropy loss as we are doing image classification (cat vs non-cat)

In [307]:
# loss function
criterion = torch.nn.BCELoss()

# 8. Gradient Descent

Next step is to define the optimizer we will be using for training the neural net. We will use gradient descent (full-batch) as out optimizer.

In [308]:
# optimizer
optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate) 

# 9. Training phase

Now we will be training the neural network to get the optimal set of weights and biases required for this problem.

In [309]:
# training phase
for epoch in range(0, num_epochs):
    
    # forward step
    pred_output = net(train_input)
    
    # find loss
    loss = criterion(pred_output.squeeze(), train_output)
    
    # backpropagation step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if((epoch + 1)%100 == 0):
        print('Loss after iteration {}: {:.4f}' .format(epoch + 1, loss.item()))

Loss after iteration 100: 0.5434
Loss after iteration 200: 0.4646
Loss after iteration 300: 0.4106
Loss after iteration 400: 0.3601
Loss after iteration 500: 0.2897
Loss after iteration 600: 0.2229
Loss after iteration 700: 0.2167
Loss after iteration 800: 0.1234
Loss after iteration 900: 0.0873
Loss after iteration 1000: 0.0678
Loss after iteration 1100: 0.0540
Loss after iteration 1200: 0.0447
Loss after iteration 1300: 0.0379
Loss after iteration 1400: 0.0324
Loss after iteration 1500: 0.0281
Loss after iteration 1600: 0.0247
Loss after iteration 1700: 0.0220
Loss after iteration 1800: 0.0197
Loss after iteration 1900: 0.0178
Loss after iteration 2000: 0.0161
Loss after iteration 2100: 0.0147
Loss after iteration 2200: 0.0135
Loss after iteration 2300: 0.0125
Loss after iteration 2400: 0.0116
Loss after iteration 2500: 0.0108
Loss after iteration 2600: 0.0100
Loss after iteration 2700: 0.0094
Loss after iteration 2800: 0.0088
Loss after iteration 2900: 0.0083
Loss after iteration 30

# 10. Testing Phase

Evaluating model on testing data

In [310]:
# testing phase
net.eval()
pred_output = net(test_input)
loss = criterion(pred_output.squeeze(), test_output)
#print("Testing Loss: " + str(loss.item()))

# accuracy
correct = 0
for index in range(0, len(pred_output)):
    if(pred_output[index] > 0.5):
        pred_output[index] = 1
    else:
        pred_output[index] = 0
    
    if(pred_output[index] == test_output[index]):
        correct = correct + 1
print("Testing accuracy is: " + str(100.0 * float(float(correct) / len(pred_output))) + "%")

Testing accuracy is: 80.0%


# 11. Results

This section contains all the hyper-parameters I tried and the corresponding accuracies.

1. learning_rate = 0.01, num_hidden_neurons = 70, num_epochs = 3000, Testing Accuracy = 76%
2. learning_rate = 0.01, num_hidden_neurons = 60, num_epochs = 3000, Testing Accuracy = 74%
3. learning_rate = 0.01, num_hidden_neurons = 50, num_epochs = 3000, Testing Accuracy = 78%
4. learning_rate = 0.01, num_hidden_neurons = 50, num_epochs = 4000, Testing Accuracy = 72%
5. learning_rate = 0.01, num_hidden_neurons = 40, num_epochs = 3000, Testing Accuracy = 80%
6. learning_rate = 0.01, num_hidden_neurons = 30, num_epochs = 3000, Testing Accuracy = 78%
7. learning_rate = 0.01, num_hidden_neurons = 30, num_epochs = 4000, Testing Accuracy = 74%
8. learning_rate = 0.01, num_hidden_neurons = 20, num_epochs = 3000, Testing Accuracy = 78%
9. learning_rate = 0.01, num_hidden_neurons = 10, num_epochs = 3000, Testing Accuracy = 68%
10. learning_rate = 0.03, num_hidden_neurons = 20, num_epochs = 3000, Testing Accuracy = 76%
11. learning_rate = 0.005, num_hidden_neurons = 20, num_epochs = 3000, Testing Accuracy = 78%
12. learning_rate = 0.005, num_hidden_neurons = 30, num_epochs = 3000, Testing Accuracy = 76%
13. learning_rate = 0.005, num_hidden_neurons = 40, num_epochs = 3000, Testing Accuracy = 74%
14. learning_rate = 0.005, num_hidden_neurons = 50, num_epochs = 3000, Testing Accuracy = 76%
15. learning_rate = 0.05, num_hidden_neurons = 50, num_epochs = 3000, Testing Accuracy = 68%
16. learning_rate = 0.05, num_hidden_neurons = 60, num_epochs = 3000, Testing Accuracy = 78%
17. learning_rate = 0.03, num_hidden_neurons = 60, num_epochs = 3000, Testing Accuracy = 76%
18. learning_rate = 0.07, num_hidden_neurons = 60, num_epochs = 3000, Testing Accuracy = 72%
19. learning_rate = 0.05, num_hidden_neurons = 40, num_epochs = 3000, Testing Accuracy = 74%
20. learning_rate = 0.05, num_hidden_neurons = 30, num_epochs = 3000, Testing Accuracy = 68%

# 12. Best Hyper-parameters obtained

The best hyper-parameters obtained were as follows: <br>
<b>learning_rate = 0.01, num_hidden_neurons = 40, num_epochs = 3000, Testing Accuracy = 80%</b>

# Predicting sentiment of movie reviews

# 13. Loading data

Using the load_data function of hw2.ipynb and then preprocessing the data as done in the hw2.ipynb notebook

In [326]:
def load_data(train_file, test_file):
    train_dataset = []
    test_dataset = []
    
    # Read the training dataset file line by line
    for line in open(train_file, 'r'):
        train_dataset.append(line.strip())
        
    for line in open(test_file, 'r'):
        test_dataset.append(line.strip())
    return train_dataset, test_dataset

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    return reviews

# loading data
train_file = "data/train_imdb.txt"
test_file = "data/test_imdb.txt"
train_dataset, test_dataset = load_data(train_file, test_file)
y = [1 if i < len(train_dataset)*0.5 else 0 for i in range(len(train_dataset))]

# pre-processing
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "
train_dataset_clean = preprocess_reviews(train_dataset)
test_dataset_clean = preprocess_reviews(test_dataset)

# Vectorization
cv = CountVectorizer(binary=True, stop_words="english", max_features=2000)
cv.fit(train_dataset_clean)
X = cv.transform(train_dataset_clean)
X_test = cv.transform(test_dataset_clean)
X = np.array(X.todense()).astype(float)
X_test = np.array(X_test.todense()).astype(float)
y = np.array(y)

# 14. Splitting of dataset

Using sklearn for splitting dataset into training and testing

In [335]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.80)
y_train = y_train.reshape(1,-1)
y_val = y_val.reshape(1,-1)

# 15. Convert dataset to Tensor form

Convert the dataset to Tensor form so that it can be fed into the PyTorch neural network.

In [336]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# use torch.from_numpy() to get the tensor form of the numpy array
train_input = torch.from_numpy(X_train).float().to(device)
train_output = torch.from_numpy(y_train).float().to(device)
train_output = train_output.squeeze()
test_input = torch.from_numpy(X_val).float().to(device)
test_output = torch.from_numpy(y_val).float().to(device)
test_output = test_output.squeeze()

In [337]:
print(train_input.shape)

torch.Size([800, 2000])


# 16. Hyper-parameters

Set the hyper-parameters for the network

In [356]:
learning_rate = 0.005
num_hidden_neurons = 200
num_epochs = 3000

# 17. Model Architecture

The model-architecture is defined using pytorch Net class. The __init__ function is where we define the architecture of the neural network, i.e in this it is two layers. The forward function is where the forward pass step of the neural network takes place.

In [357]:
# neural network class
class Net(torch.nn.Module):
    # init function
    def __init__(self, num_input_neurons, num_hidden_neurons, num_output_neurons):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(num_input_neurons, num_hidden_neurons)
        self.fc2 = torch.nn.Linear(num_hidden_neurons, num_output_neurons)
        
    # forward pass step of the neural network
    def forward(self, input):
        output = torch.nn.functional.sigmoid(self.fc2(torch.nn.functional.relu(self.fc1(input))))
        return output
    
# get the neural net object
net = Net(int(train_input.shape[1]), int(num_hidden_neurons), 1).to(device)
print(net)

Net(
  (fc1): Linear(in_features=2000, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=1, bias=True)
)


# 18. Loss function

We will use Binary Cross-entropy loss as we are doing sentiment analysis

In [358]:
# loss function
criterion = torch.nn.BCELoss()

# 19. Gradient Descent

Next step is to define the optimizer we will be using for training the neural net. We will use gradient descent (full-batch) as out optimizer.

In [359]:
# optimizer
optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate) 

# 20. Training phase

Now we will be training the neural network to get the optimal set of weights and biases required for this problem.

In [360]:
# training phase
for epoch in range(0, num_epochs):
    
    # forward step
    pred_output = net(train_input)
    
    # find loss
    loss = criterion(pred_output.squeeze(), train_output)
    
    # backpropagation step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if((epoch + 1)%100 == 0):
        print('Loss after iteration {}: {:.4f}' .format(epoch + 1, loss.item()))

Loss after iteration 100: 0.6894
Loss after iteration 200: 0.6855
Loss after iteration 300: 0.6814
Loss after iteration 400: 0.6769
Loss after iteration 500: 0.6718
Loss after iteration 600: 0.6661
Loss after iteration 700: 0.6594
Loss after iteration 800: 0.6517
Loss after iteration 900: 0.6427
Loss after iteration 1000: 0.6321
Loss after iteration 1100: 0.6198
Loss after iteration 1200: 0.6058
Loss after iteration 1300: 0.5898
Loss after iteration 1400: 0.5721
Loss after iteration 1500: 0.5528
Loss after iteration 1600: 0.5321
Loss after iteration 1700: 0.5104
Loss after iteration 1800: 0.4881
Loss after iteration 1900: 0.4655
Loss after iteration 2000: 0.4432
Loss after iteration 2100: 0.4213
Loss after iteration 2200: 0.4002
Loss after iteration 2300: 0.3799
Loss after iteration 2400: 0.3606
Loss after iteration 2500: 0.3423
Loss after iteration 2600: 0.3250
Loss after iteration 2700: 0.3087
Loss after iteration 2800: 0.2934
Loss after iteration 2900: 0.2790
Loss after iteration 30

# 21. Testing phase

Evaluating model on testing data

In [361]:
# testing phase
net.eval()
pred_output = net(test_input)
loss = criterion(pred_output.squeeze(), test_output)
#print("Testing Loss: " + str(loss.item()))

# accuracy
correct = 0
for index in range(0, len(pred_output)):
    if(pred_output[index] > 0.5):
        pred_output[index] = 1
    else:
        pred_output[index] = 0
    
    if(pred_output[index] == test_output[index]):
        correct = correct + 1
print("Testing accuracy is: " + str(100.0 * float(float(correct) / len(pred_output))) + "%")

Testing accuracy is: 80.5970149254%


# 22. Results

This section contains all the hyper-parameters I tried and the corresponding accuracies.

1. learning_rate = 0.01, num_hidden_neurons = 70, num_epochs = 3000, Testing Accuracy = 76%
2. learning_rate = 0.01, num_hidden_neurons = 60, num_epochs = 3000, Testing Accuracy = 74%
3. learning_rate = 0.01, num_hidden_neurons = 50, num_epochs = 3000, Testing Accuracy = 78%
4. learning_rate = 0.01, num_hidden_neurons = 50, num_epochs = 4000, Testing Accuracy = 72%
5. learning_rate = 0.01, num_hidden_neurons = 40, num_epochs = 3000, Testing Accuracy = 80%
6. learning_rate = 0.01, num_hidden_neurons = 30, num_epochs = 3000, Testing Accuracy = 78%
7. learning_rate = 0.01, num_hidden_neurons = 30, num_epochs = 4000, Testing Accuracy = 74%
8. learning_rate = 0.01, num_hidden_neurons = 20, num_epochs = 3000, Testing Accuracy = 78%
9. learning_rate = 0.01, num_hidden_neurons = 10, num_epochs = 3000, Testing Accuracy = 68%
10. learning_rate = 0.03, num_hidden_neurons = 20, num_epochs = 3000, Testing Accuracy = 76%
11. learning_rate = 0.005, num_hidden_neurons = 20, num_epochs = 3000, Testing Accuracy = 78%
12. learning_rate = 0.005, num_hidden_neurons = 30, num_epochs = 3000, Testing Accuracy = 76%
13. learning_rate = 0.005, num_hidden_neurons = 40, num_epochs = 3000, Testing Accuracy = 74%
14. learning_rate = 0.005, num_hidden_neurons = 50, num_epochs = 3000, Testing Accuracy = 76%
15. learning_rate = 0.05, num_hidden_neurons = 50, num_epochs = 3000, Testing Accuracy = 68%
16. learning_rate = 0.05, num_hidden_neurons = 60, num_epochs = 3000, Testing Accuracy = 78%
17. learning_rate = 0.03, num_hidden_neurons = 60, num_epochs = 3000, Testing Accuracy = 76%
18. learning_rate = 0.07, num_hidden_neurons = 60, num_epochs = 3000, Testing Accuracy = 72%
19. learning_rate = 0.05, num_hidden_neurons = 40, num_epochs = 3000, Testing Accuracy = 74%
20. learning_rate = 0.05, num_hidden_neurons = 30, num_epochs = 3000, Testing Accuracy = 68%

# 23. Best Hyper-parameters obtained

The best hyper-parameters obtained were as follows: <br>
<b>learning_rate = 0.01, num_hidden_neurons = 40, num_epochs = 3000, Testing Accuracy = 80%</b>